In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tkinter as tk
from tkinter import messagebox

def train_drowsiness_model(epoch,batch_size):
    try:
        print("Prepare Dataset..")

        labels = os.listdir(r"src/dataset/train")

        print(labels)

        import matplotlib.pyplot as plt
       

        a = plt.imread(r"src/dataset/train/yawn/10.jpg")

        print(a.shape)

     

        def face_for_yawn(direc=r"src/dataset/train", face_cas_path=r"src/data/haarcascade_frontalface_default.xml"):
            yaw_no = []
            IMG_SIZE = 145
            categories = ["yawn", "no_yawn"]
            for category in categories:
                path_link = os.path.join(direc, category)
                class_num1 = categories.index(category)
                print(class_num1)
                for image in os.listdir(path_link):
                    image_array = cv2.imread(os.path.join(path_link, image), cv2.IMREAD_COLOR)
                    face_cascade = cv2.CascadeClassifier(face_cas_path)
                    faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
                    for (x, y, w, h) in faces:
                        img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        roi_color = img[y:y+h, x:x+w]
                        resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                        yaw_no.append([resized_array, class_num1])
            return yaw_no

        print("training face_for_yawn..")
        yawn_no_yawn = face_for_yawn()

        def get_data(dir_path=r"src/dataset/train", face_cas=r"src/data/haarcascade_frontalface_default.xml", eye_cas=r"src/data/haarcascade.xml"):
            labels = ['Closed', 'Open']
            IMG_SIZE = 145
            data = []
            for label in labels:
                path = os.path.join(dir_path, label)
                class_num = labels.index(label)
                class_num +=2
                print(class_num)
                for img in os.listdir(path):
                    try:
                        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                        resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                        data.append([resized_array, class_num])
                    except Exception as e:
                        print(e)
            return data

        print("get data..")

        data_train = get_data()

        def append_data():
            yaw_no = face_for_yawn()
            data = get_data()

            # Ensure all elements in yaw_no have the same shape
            yaw_no = [(np.array(item[0]), item[1]) for item in yaw_no]
            data = [(np.array(item[0]), item[1]) for item in data]

            # Combine the data
            combined_data = yaw_no + data

            # Create a NumPy array with dtype as object
            return np.array(combined_data, dtype=object)


        print("process append_data..")
        new_data = append_data()

        X = []
        y = []

        for feature, label in new_data:
            X.append(feature)
            y.append(label)

        X = np.array(X)
        X = X.reshape(-1, 145, 145, 3)

        from sklearn.preprocessing import LabelBinarizer
        label_bin = LabelBinarizer()
        y = label_bin.fit_transform(y)

        y = np.array(y)

        from sklearn.model_selection import train_test_split
        seed = 42
        test_size = 0.30
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

        len(X_test)

        from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
        from tensorflow.keras.models import Model
        from tensorflow.keras.models import Sequential
        from keras.preprocessing.image import ImageDataGenerator
        import tensorflow as tf

        print("process Train generator")
        train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
        test_generator = ImageDataGenerator(rescale=1/255)


        #train_generator = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        #test_generator = tf.data.Dataset.from_tensor_slices((X_test, y_test))

        train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
        test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

        print("process model")
        model = Sequential()

        model.add(Conv2D(256, (3, 3), activation="relu", input_shape=(145,145,3)))
        model.add(MaxPooling2D(2, 2))

        model.add(Conv2D(128, (3, 3), activation="relu"))
        model.add(MaxPooling2D(2, 2))

        model.add(Conv2D(64, (3, 3), activation="relu"))
        model.add(MaxPooling2D(2, 2))

        model.add(Conv2D(32, (3, 3), activation="relu"))
        model.add(MaxPooling2D(2, 2))

        model.add(Flatten())
        model.add(Dropout(0.5))

        model.add(Dense(64, activation="relu"))
        model.add(Dense(4, activation="softmax"))

        model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

        print("Model Summary")
        model.summary()

        print("Training Model")
        history = model.fit(train_generator, epochs=epoch,batch_size=batch_size, validation_data=test_generator, shuffle=True, validation_steps=len(test_generator))

        accuracy = history.history['accuracy']
        val_accuracy = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(accuracy))

#         plt.plot(epochs, accuracy, "b", label="trainning accuracy")
#         plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
#         plt.legend()
#         plt.show()

#         plt.plot(epochs, loss, "b", label="trainning loss")
#         plt.plot(epochs, val_loss, "r", label="validation loss")
#         plt.legend()
#         plt.show()

        model.save("src/model.h5")

        print("model saved")
        messagebox.showinfo("Train Data Success", f"Train data Success")
        
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

In [11]:
import cv2
import numpy as np
import tkinter as tk
import tkinter.font as tkFont
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import PhotoImage
import tkinter.messagebox as messagebox
import shutil
import os
import subprocess
import time

class ComputerVisionApp:
    def __init__(self, master):
        self.master = master
        self.master.title("UAS Computer Vision App - Drowsines Detection")
        
        #  check folder
        self.check_folder()
        # Create a frame as a container for the buttons
        self.button_frame = tk.Frame(self.master)
        self.button_frame.pack(pady=200) 
        # Load the image
        self.image = Image.open('./image/car.png')  
        self.photo = ImageTk.PhotoImage(self.image)

        # Create a label to display the image
        self.image_label = tk.Label(self.master, image=self.photo)
        self.image_label.place(x=170,y=400) 
       
        
    
        self.load_icon = PhotoImage(file='./image/icon.png')  
        self.quit_icon = PhotoImage(file='./image/quit.png')
        self.detection_icon = PhotoImage(file='./image/detection.png')
        
        self.load_button = tk.Button( self.button_frame, text="Train Data", command=self.open_train_data,  compound=tk.TOP, image=self.load_icon, padx=10, pady=5)
        self.load_button.pack(side=tk.LEFT,padx=20)

        self.process_button = tk.Button( self.button_frame, text="Detection", command=self.process_image,  compound=tk.TOP, image=self.detection_icon, padx=10, pady=5)
        self.process_button.pack(side=tk.LEFT,padx=20)

        self.quit_button = tk.Button( self.master, text="Quit", command=self.master.destroy,  compound=tk.TOP, image=self.quit_icon, padx=10, pady=5)
        self.quit_button.pack(padx=20)
        self.quit_button.place(x=520,y=10)
        # Initialize variables
        self.image_path = None
        self.cv_image = None

    def load_image(self):
        self.image_path = filedialog.askopenfilename()
        if self.image_path:
            self.cv_image = cv2.imread(self.image_path)
            self.show_image(self.cv_image)

    def process_image(self):
        file_path = './src/model.h5'

        if os.path.exists(file_path):
             !python ./src/detect_drowsiness.py 
        else:
            messagebox.showerror("File Not Found", "The 'model.h5' file does not exist in the '/src/' directory. Please check.")
       
    

    def show_image(self, image):
        # Convert the OpenCV image to a format that Tkinter can display
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)

        # Update the image label
        self.image_label.configure(image=image, compound=tk.BOTTOM, text="Your text here")
        self.image_label.image = image
        
    def open_train_data(self):
        self.master.withdraw()  # Hide the main window

        self.new_window = tk.Toplevel(self.master)
        self.new_window.title("Train Data")
        self.new_window.geometry("600x500")
        self.new_window.protocol("WM_DELETE_WINDOW", lambda: self.on_close_train_window())  # Event when new window is closed

        # Add an image to the train window
        train_image = Image.open('./image/driver.png')
        train_photo = ImageTk.PhotoImage(train_image)

        train_image_label = tk.Label(self.new_window, image=train_photo)
        train_image_label.image = train_photo
        train_image_label.place(relx=1, rely=1, anchor='se')

#         button select file
        button_file=tk.Button(self.new_window)
        button_file["bg"] = "#33eb0e"
        ft = tkFont.Font(family='Times',size=10)
        button_file["font"] = ft
        button_file["fg"] = "#000000"
        button_file["justify"] = "center"
        button_file["text"] = "Select Folder"
        button_file.place(x=70,y=50,width=150,height=25)
        button_file["command"] = self.select_folder
        
        self.entry_epoch=tk.Entry(self.new_window)
        self.entry_epoch["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.entry_epoch["font"] = ft
        self.entry_epoch["fg"] = "#333333"
        self.entry_epoch["justify"] = "center"
        self.entry_epoch["text"] = "Entry"
        self.entry_epoch.place(x=160,y=110,width=70,height=25)
        # Validate batch size entry
        self.entry_epoch.config(validate="key")
        self.entry_epoch['validatecommand'] = (self.entry_epoch.register(self.validate_batch_size), '%P')
        
        label_epoch=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        label_epoch["font"] = ft
        label_epoch["fg"] = "#333333"
        label_epoch["justify"] = "center"
        label_epoch["text"] = "Epoch"
        label_epoch.place(x=60,y=110,width=70,height=25)

        label_batch=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        label_batch["font"] = ft
        label_batch["fg"] = "#333333"
        label_batch["justify"] = "center"
        label_batch["text"] = "Batch Size"
        label_batch.place(x=60,y=160,width=70,height=25)

        self.entry_batch=tk.Entry(self.new_window)
        self.entry_batch["borderwidth"] = "1px"
        ft = tkFont.Font(family='Times',size=10)
        self.entry_batch["font"] = ft
        self.entry_batch["fg"] = "#333333"
        self.entry_batch["justify"] = "center"
        self.entry_batch["text"] = "Entry Batch Size"
        self.entry_batch.place(x=160,y=160,width=70,height=25)
        # Validate batch size entry
        self.entry_batch.config(validate="key")
        self.entry_batch['validatecommand'] = (self.entry_batch.register(self.validate_batch_size), '%P')
        
        
        self.file_label=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        self.file_label["font"] = ft
        self.file_label["fg"] = "#333333"
        self.file_label["justify"] = "center"
        self.file_label["text"] = "Folder not selected"
        self.file_label.place(x=200,y=50,width=300,height=25)

        btn_train=tk.Button(self.new_window)
        btn_train["bg"] = "#0eebe4"
        ft = tkFont.Font(family='Times',size=10)
        btn_train["font"] = ft
        btn_train["fg"] = "#000000"
        btn_train["justify"] = "center"
        btn_train["text"] = "Train Data"
        btn_train.place(x=270,y=220,width=70,height=25)
        btn_train["command"] = self.train_data

        btn_default=tk.Button(self.new_window)
        btn_default["bg"] = "#ffd700"
        ft = tkFont.Font(family='Times',size=10)
        btn_default["font"] = ft
        btn_default["fg"] = "#000000"
        btn_default["justify"] = "center"
        btn_default["text"] = "Use Default Model"
        btn_default.place(x=60,y=350,width=120,height=25)
        btn_default["command"] = self.model_default

        btn_back=tk.Button(self.new_window)
        btn_back["bg"] = "#ff5722"
        ft = tkFont.Font(family='Times',size=10)
        btn_back["font"] = ft
        btn_back["fg"] = "#000000"
        btn_back["justify"] = "center"
        btn_back["text"] = "Back"
        btn_back.place(x=60,y=410,width=70,height=25)
        btn_back["command"] = lambda: self.back_to_main(self.new_window)
        
       

        self.loading_label=tk.Label(self.new_window)
        ft = tkFont.Font(family='Times',size=10)
        self.loading_label["font"] = ft
        self.loading_label["fg"] = "#333333"
        self.loading_label["justify"] = "center"
        self.loading_label["text"] = ""
        self.loading_label.place(x=450,y=20,width=70,height=25)

    def select_folder(self):
        folder_path = filedialog.askdirectory()
        if folder_path:
            try:
                file_name = os.path.basename(folder_path)  # Extract folder name

                # Copy the contents of the selected folder to the src folder
                src_folder_path = './src/dataset/'
                dist_folder_path = os.path.join(src_folder_path, file_name)

                if os.path.exists(dist_folder_path):
                    shutil.rmtree(dist_folder_path)  # Remove the existing folder in src

                shutil.copytree(folder_path, dist_folder_path)
                messagebox.showinfo("Folder Updated", f"{file_name} copied to src/dataset/ ")
                self.file_label.config(text=f"Selected Folder: {file_name}")  # Update label with folder name
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")
                
                
    def validate_batch_size(self, new_text):
        if new_text.isdigit() or new_text == "":
            return True
        return False

        
    def check_folder(self):
        folder_path = './src'
        if not os.path.exists(folder_path):
            messagebox.showerror("Folder Not Found", f"The folder '{folder_path}' does not exist!")
            self.master.destroy()
   
                
    
    def train_data(self):
        train_folder = './src/dataset/train'

        if not (os.path.exists(train_folder)):
            messagebox.showerror("Folders Not Found", "The 'dataset/train'  folders do not exist.")
        else:
            messagebox.showinfo("Loading", "Train Data process...")
            
            batch_size_value = int(self.entry_batch.get())
            epoch_value = int(self.entry_epoch.get())
            
            if not batch_size_value:
                batch_size_value = 128

            if not epoch_value:
                epoch_value = 5
            
            train_drowsiness_model(epoch_value,batch_size_value)

           

            

    
    def model_default(self):
        default_model_path = './src/default/model.h5'  # Path to default model
        root_model_path = './src/model.h5'  # Root folder model path

        # Check if model exists in the default folder
        if os.path.exists(default_model_path):
            # Remove model from root folder if it exists
            if os.path.exists(root_model_path):
                os.remove(root_model_path)

            # Copy model from default folder to root folder
            shutil.copy(default_model_path, root_model_path)
            messagebox.showinfo("Model Updated", "Model updated from default model.")
        else:
            messagebox.showwarning("File Not Found", "file model.h5 not found in default folder.")
        

    def on_close_train_window(self):
        self.master.deiconify()  # Show the main window when the train window is closed

    def back_to_main(self, window):
        window.destroy()  # Close the train window
        self.master.deiconify()

def main():
    root = tk.Tk()
    root.geometry("600x600")
    root.resizable(False, False)
    app = ComputerVisionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
